In [11]:
import sys
print(sys.path)

['/usr/lib64/python36.zip', '/usr/lib64/python3.6', '/usr/lib64/python3.6/lib-dynload', '', '/home/kdevereaux/.local/lib/python3.6/site-packages', '/usr/local/lib64/python3.6/site-packages', '/usr/local/lib/python3.6/site-packages', '/usr/lib64/python3.6/site-packages', '/usr/lib/python3.6/site-packages', '/usr/local/lib/python3.6/site-packages/IPython/extensions', '/home/kdevereaux/.ipython', '/home/kdevereaux/.local/lib/python3.6/site-packages']


In [13]:
import numpy as np
import pandas as pd
import uproot as ur

from keras.layers import Dense, Input
from keras.models import Model

import omnifold as of
import os
import tensorflow as tf

from matplotlib import pyplot as plt
from IPython.display import Image
pd.set_option('display.max_columns', None) # to see all columns of df.head()

In [14]:
#gpus = tf.config.experimental.list_physical_devices('GPU')
#print(gpus)

# [Omnifold Paper](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.124.182001) Background

In [15]:
#Image(filename='Omnifold.png') 

|   |   |   |
|---|---|---|
|Synthetic Generator-Level Sim   | $\theta_{0,G}$  | Truth-Level Sim  |
|Synthetic Reconstruction-Level Sim   | $\theta_{0,S}$   | Full Reco-Level Sim  |
|Natural Reconstruction  | $\theta_\mathrm{unknown,S}$  | Observed Detector Data  |
|Natural Truth   |$\theta_\mathrm{unknown,G}$   | Nature  |


___
___

# Omnifold for EEC unfolding, (energy_weight, R_L, jet_pt)

#### we assume all imported root files consist of `TTree preprocessed` consiting of three columns `energy_weight`, `R_L`, and `jet_pt`

### Define variables to unfold

In [1]:
obs_features = ["energy_weight", "R_L", "jet_pt"]
gen_features = obs_features

labels = ["energy weight", "$R_L$", "jet $p_T$"]

### Import "natural reco": ALICE measured (testing: PYTHIA8 generator-level)

In [ ]:
natural_reco_file = "merged.root"
natural_reco_tree = ur.open("%s:Pythia/preprocessed"%(natural_reco_file))
natural_reco_df = natural_reco_tree.arrays(library="pd") #open the TTree as a pandas data frame
natural_reco_df = natural_reco_df[natural_reco_df["obs_hfs_pt"] != 0] #remove empty entries

### Import "natural truth": ALICE truth (testing: PYTHIA8 generator-level, unused during actuall unfolding) 

In [ ]:
natural_truth_file = "all-h1-django.root"
natural_truth_tree = ur.open("%s:Django/minitree"%(natural_reco_file))
natural_truth_df = natural_reco_tree.arrays(library="pd") #open the TTree as a pandas data frame
natural_reco_df = natural_reco_df[natural_reco_df["obs_hfs_pt"] != 0] #remove empty entries

### Take a quick look at the data

In [ ]:
natural_df.describe()

## Import "Synthetic simulation" [Rapgap], both generated and reconstructed level.

In [ ]:
synthetic_file = "all-h1-rapgap.root"
synth_tree = ur.open("%s:Rapgap/minitree"%(synthetic_file))
synth_df = synth_tree.arrays(library="pd")
synth_df = synth_df[synth_df["obs_hfs_pt"] != 0]

In [ ]:
synth_df.head(3) #look at 3 entries

### define 4 main datasets

In [ ]:
theta_unknown_S = natural_df[obs_features].to_numpy() #Reconstructed Data
theta_unknown_G = natural_df[gen_features].to_numpy() #Nature, which unfolded data approaches

theta0_S = synth_df[obs_features].to_numpy() #Simulated, synthetic reco-level
theta0_G = synth_df[gen_features].to_numpy() #Generated, synthetic truth-level

### Ensure the samples have the same number of entries

In [ ]:
N_Events = min(np.shape(theta0_S)[0],np.shape(theta_unknown_S)[0])-1

theta0_S = theta0_S[:N_Events]
theta0_G = theta0_G[:N_Events]
theta_unknown_S = theta_unknown_S[:N_Events]
theta_unknown_G = theta_unknown_G[:N_Events]

In [ ]:
N = len(obs_features)

binning = [np.linspace(0,80,100),np.linspace(-2,6,100),
           np.linspace(10,80,100),np.linspace(-10,40,100),np.linspace(-2,2,100)]

fig, axes = plt.subplots(2, 3, figsize=(15,7))

for i,ax in enumerate(axes.ravel()):
    if (i >= N): break
    _,_,_=ax.hist(theta0_G[:,i],binning[i],color='blue',alpha=0.5,label="MC, true")
    _,_,_=ax.hist(theta0_S[:,i],binning[i],histtype="step",color='black',ls=':',label="MC, reco")
    _,_,_=ax.hist(theta_unknown_G[:,i],binning[i],color='orange',alpha=0.5,label="Data, true")
    _,_,_=ax.hist(theta_unknown_S[:,i],binning[i],histtype="step",color='black',label="Data, reco")

    ax.set_title(labels[i])
    ax.set_xlabel(labels[i])
    ax.set_ylabel("Events")
    ax.legend(frameon=False)
    
fig.tight_layout()

In [ ]:
inputs = Input((len(obs_features), ))
hidden_layer_1 = Dense(50, activation='relu')(inputs)
hidden_layer_2 = Dense(50, activation='relu')(hidden_layer_1)
hidden_layer_3 = Dense(50, activation='relu')(hidden_layer_2)
outputs = Dense(1, activation='sigmoid')(hidden_layer_3)
model_dis = Model(inputs=inputs, outputs=outputs)

In [ ]:
N_Iterations = 2

myweights = of.omnifold(theta0_G,theta0_S,theta_unknown_S,N_Iterations,model_dis)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15,7))

for i,ax in enumerate(axes.ravel()):
    if (i >= N): break
    _,_,_=ax.hist(theta0_G[:,i],binning[i],color='blue',alpha=0.5,label="MC, true")
    _,_,_=ax.hist(theta_unknown_G[:,i],binning[i],color='orange',alpha=0.5,label="Data, true")
    _,_,_=ax.hist(theta0_G[:,i],weights=myweights[-1, 0, :],bins=binning[i],color='black',histtype="step",label="OmniFolded",lw="2")

    ax.set_title(labels[i])
    ax.set_xlabel(labels[i])
    ax.set_ylabel("Events")
    ax.legend(frameon=False)
    
fig.tight_layout()

___
___

# OmniFold: DIY

### 1. Add Features, or create your own. Add iterations to the unfolding
careful adding both: might take a while!

### 2. Plot _unfolded_ electron observables: $\Sigma_e = E_e - p_{z,e}, \ \ \theta = 2\arctan(e^{-\eta})$

hint: unfolded distributions are just weighted histograms

### 3. Plot at _unfolded_ x,y, and $Q^2$ distribution

Hint: $y = 1-\frac{\Sigma_e}{2E_0}, \ \ Q^2 = \frac{E^2sin^2\theta}{1-y}, \ \ x\cdot E_p = \frac{E(1+\cos\theta)}{2y}$